In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import tensorflow as tf
from tensorflow import keras

In [2]:
# Download dataset
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file(
    'flower_photos', origin=dataset_url, cache_dir=".", untar=True)


In [3]:
data_dir

'./datasets/flower_photos'

In [4]:
import pathlib
data_dir = pathlib.Path(data_dir)
data_dir

PosixPath('datasets/flower_photos')

In [5]:
roses_list = list(data_dir.glob('roses/*'))
roses_list[:5]

[PosixPath('datasets/flower_photos/roses/5360769702_ec28c53b9e_n.jpg'),
 PosixPath('datasets/flower_photos/roses/12238827553_cf427bfd51_n.jpg'),
 PosixPath('datasets/flower_photos/roses/7502389724_85b4a6c855_n.jpg'),
 PosixPath('datasets/flower_photos/roses/12395698413_c0388278f7.jpg'),
 PosixPath('datasets/flower_photos/roses/15699509054_d3e125286f_n.jpg')]

In [6]:
#  read flowers from disk into numpy array using openCV

flowers_image_dict = {
    'roses': list(data_dir.glob("roses/*")),
    'daisy': list(data_dir.glob('daisy/*')),
    'dandelion': list(data_dir.glob('dandelion/*')),
    'sunflowers': list(data_dir.glob('sunflowers/*')),
    'tulips': list(data_dir.glob('tulips/*')),
}

flowers_label_dict = {
    'roses': 0,
    'daisy': 1,
    'dandelion': 2,
    'sunflowers': 3,
    'tulips': 4,
}


In [7]:
# create your X and y
X,y = [],[]

for flower_name, images in flowers_image_dict.items():
    for image in images:
        img = cv2.imread(str(image))
        resized = cv2.resize(img, (180,180))
        X.append(resized)
        y.append(flowers_label_dict[flower_name])

X = np.array(X)
y = np.array(y)




In [8]:
# train, test, split
from sklearn.model_selection import train_test_split

X_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [9]:
# scaling

X_train = X_train / 255
x_test = x_test / 255

In [10]:
num_classes = 5

data_augmentation = keras.Sequential(
  [
    keras.layers.experimental.preprocessing.RandomFlip("horizontal", 
                                                 input_shape=(180, 
                                                              180,
                                                              3)),
    keras.layers.experimental.preprocessing.RandomRotation(0.1),
    keras.layers.experimental.preprocessing.RandomZoom(0.1),
  ]
)

cnn = keras.models.Sequential([
data_augmentation,
  keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
  keras.layers.MaxPooling2D(),
  keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
  keras.layers.MaxPooling2D(),
  keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
  keras.layers.MaxPooling2D(),
  keras.layers.Dropout(0.2),
  keras.layers.Flatten(),
  keras.layers.Dense(128, activation='relu'),
  keras.layers.Dense(num_classes)
])

cnn.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [11]:
cnn.fit(X_train, y_train, epochs=10)

Epoch 1/10
92/92 [==============================] - 88s 950ms/step - loss: 1.4005 - accuracy: 0.3767
Epoch 2/10
92/92 [==============================] - 88s 954ms/step - loss: 1.1061 - accuracy: 0.5480
Epoch 3/10
92/92 [==============================] - 87s 944ms/step - loss: 1.0158 - accuracy: 0.6015
Epoch 4/10
92/92 [==============================] - 87s 942ms/step - loss: 0.9340 - accuracy: 0.6434
Epoch 5/10
92/92 [==============================] - 85s 928ms/step - loss: 0.8623 - accuracy: 0.6652
Epoch 6/10
92/92 [==============================] - 85s 922ms/step - loss: 0.8086 - accuracy: 0.6914
Epoch 7/10
92/92 [==============================] - 85s 922ms/step - loss: 0.7813 - accuracy: 0.7030
Epoch 8/10
92/92 [==============================] - 86s 940ms/step - loss: 0.7421 - accuracy: 0.7132
Epoch 9/10
92/92 [==============================] - 86s 932ms/step - loss: 0.6968 - accuracy: 0.7282
Epoch 10/10
92/92 [==============================] - 85s 926ms/step - loss: 0.6762 - accura

In [13]:
cnn.evaluate(x_test, y_test)

23/23 [==============================] - 6s 237ms/step - loss: 0.7708 - accuracy: 0.7125


[0.7708163261413574, 0.7125340700149536]

In [17]:
y_predict = cnn.predict(x_test)


In [24]:
y_answer = [np.argmax(elements) for elements in y_predict]
y_answer[:10]

[4, 0, 4, 4, 4, 3, 0, 4, 3, 4]

In [23]:
y_test[:10]

array([1, 0, 2, 4, 2, 3, 0, 4, 3, 4])